In [1]:
#reading a text file
with open('/workspace/llm-chatbot-python/knowledge_graph_generation/bible.txt', 'r', encoding='ascii', errors='ignore') as file:
    text = file.read()

In [2]:
import streamlit as st
import os
from langchain_community.graphs import Neo4jGraph


NEO4J_URI = st.secrets["NEO4J_URI"]
NEO4J_USERNAME = st.secrets["NEO4J_USERNAME"]
NEO4J_PASSWORD = st.secrets["NEO4J_PASSWORD"]
NEO4J_DATABASE = st.secrets["NEO4J_DATABASE"]

graph = Neo4jGraph(
    url=NEO4J_URI,
    username=NEO4J_USERNAME,
    password=NEO4J_PASSWORD,
    database=NEO4J_DATABASE
)

In [3]:
from transformers import pipeline

def perform_ner(text):
    pipe = pipeline("token-classification", model="dslim/bert-base-NER")
    results = pipe(text)

    entities = []
    for entity in results:
        entities.append({
            "entity": entity["entity"],
            "word": entity["word"],
        })

    return entities

/workspace/.pyenv_mirror/user/current/lib/python3.12/site-packages/tqdm/auto.py:21: TqdmWarning: IProgress not found. Please update jupyter and ipywidgets. See https://ipywidgets.readthedocs.io/en/stable/user_install.html
  from .autonotebook import tqdm as notebook_tqdm


In [4]:
def embed_verse(text):
    pipe = pipeline("feature-extraction", model="nvidia/NV-Embed-v2", trust_remote_code=True)
    embeddings = pipe(text)
    
    return embeddings

In [5]:
import json
import re
import transformers
transformers.logging.set_verbosity_error()

def preprocess_bible_text(text):
    lines = text.split('\n')[2:]
    structured_text = []
    current_book = ""
    current_chapter = ""
    
    for line in lines:
        match = re.match(r'(\w+)\s+(\d+):(\d+)\s+(.+)', line)
        if match:
            book, chapter, verse, content = match.groups()
            if book != current_book:
                structured_text.append({"type": "Book", "name": book, "chapters": []})
                current_book = book
            if chapter != current_chapter:
                structured_text[-1]["chapters"].append({"number": int(chapter), "verses": []})
                current_chapter = chapter
            if structured_text and structured_text[-1]["chapters"]:
                structured_text[-1]["chapters"][-1]["verses"].append({
                    "number": int(verse),
                    "text": content,
                    "entities": perform_ner(content),
                    "embeddings": embed_verse(content)
                })
    return structured_text

preprocessed_text = preprocess_bible_text(text)

with open('bible_structured.json', 'w') as f:
    json.dump(preprocessed_text, f)


/workspace/.pyenv_mirror/user/current/lib/python3.12/site-packages/transformers/tokenization_utils_base.py:1601: FutureWarning: `clean_up_tokenization_spaces` was not set. It will be set to `True` by default. This behavior will be depracted in transformers v4.45, and will be then set to `False` by default. For more details check this issue: https://github.com/huggingface/transformers/issues/31884
  warnings.warn(


AttributeError: 'LatentAttentionConfig' object has no attribute '_attn_implementation_internal'